# CSV Legal Dataset → RAG-based Personal Law Assistant

This notebook converts a CSV legal dataset into a smart legal assistant using RAG (Retrieval Augmented Generation). No model training required - we'll use pre-trained embeddings and LLMs.

In [1]:
# Step 1: Install Required Packages
!pip install -U langchain langchain-community langchain-ollama chromadb sentence-transformers pandas streamlit numpy

# Check if installations were successful
try:
    import langchain
    import chromadb
    print("✅ All packages installed successfully!")
    print(f"LangChain version: {langchain.__version__}")
except ImportError as e:
    print(f"❌ Installation issue: {e}")
    print("Please restart the kernel and run again")

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 14.0 MB/s  0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 22.7 MB/s  0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 13.7 MB/s  0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 13.5 MB/s  0:00:00
   ---------------------------------------- 0.0/21.4 MB ? eta -:--:--
   ----- ---------------------------------- 3.1/21.4 MB 17.8 MB/s eta 0:00:02
   ------------- -------------------------- 7.1/21.4 MB 18.7 MB/s eta 0:00:01
   -------------------- ------------------- 11.0/21.4 MB 18.7 MB/s eta 0:00:01
   ---------------------------- ----------- 15.5/21.4 MB 19.8 MB/s eta 0:00:01
   -------------------------------- ------- 


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ All packages installed successfully!
LangChain version: 1.2.8


In [4]:
# Step 2: Import Required Libraries
import pandas as pd
import numpy as np
import os
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    
try:
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.vectorstores import Chroma
except ImportError:
    from langchain.embeddings import HuggingFaceEmbeddings
    from langchain.vectorstores import Chroma

try:
    from langchain_core.prompts import PromptTemplate
    from langchain_core.documents import Document
except ImportError:
    try:
        from langchain.prompts import PromptTemplate
        from langchain.schema import Document
    except ImportError:
        # Create simple alternatives
        class PromptTemplate:
            def __init__(self, input_variables=None, template=""):
                self.input_variables = input_variables or []
                self.template = template
            
            @classmethod
            def from_template(cls, template):
                return cls(template=template)
        
        class Document:
            def __init__(self, page_content="", metadata=None):
                self.page_content = page_content
                self.metadata = metadata or {}

try:
    from langchain.chains import RetrievalQA
except ImportError:
    # We'll create a simple alternative later
    RetrievalQA = None

import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [5]:
# Step 3: Create Sample Legal Dataset (or load your CSV)

# Option 1: Create sample legal data for demonstration
sample_legal_data = [
    {
        'case_name': 'State vs. Kumar',
        'ipc_section': 'IPC 420 - Cheating',
        'judgment_summary': 'Defendant found guilty of cheating and dishonestly inducing delivery of property. Sentenced to 2 years imprisonment and fine of Rs. 10,000.',
        'citizen_rights': 'Right to legal representation, right to bail application, right to appeal within 30 days',
        'category': 'Criminal Law',
        'year': 2023
    },
    {
        'case_name': 'Consumer vs. XYZ Company',
        'ipc_section': 'Consumer Protection Act 2019',
        'judgment_summary': 'Company found liable for deficiency in service. Ordered to refund amount plus compensation.',
        'citizen_rights': 'Right to file complaint in consumer forum, right to compensation for deficiency in service',
        'category': 'Consumer Rights',
        'year': 2024
    },
    {
        'case_name': 'Arrest Rights Case',
        'ipc_section': 'CrPC Section 50, 41, 41A',
        'judgment_summary': 'Supreme Court guidelines on arrest procedures. Police must follow mandatory procedures during arrest.',
        'citizen_rights': 'Right to know grounds of arrest, right to legal aid, right to inform family, right against torture',
        'category': 'Arrest Rights',
        'year': 2023
    },
    {
        'case_name': 'Domestic Violence Case',
        'ipc_section': 'Protection of Women from Domestic Violence Act 2005',
        'judgment_summary': 'Court granted protection order and maintenance to victim of domestic violence.',
        'citizen_rights': 'Right to protection order, right to residence order, right to maintenance and custody',
        'category': 'Women Rights',
        'year': 2024
    },
    {
        'case_name': 'Bail Application Matter',
        'ipc_section': 'CrPC Section 437, 439',
        'judgment_summary': 'Guidelines for granting bail in non-bailable offences. Court considers flight risk and evidence tampering.',
        'citizen_rights': 'Right to bail application, right to legal representation during bail hearing',
        'category': 'Bail Rights',
        'year': 2023
    }
]

# Create DataFrame
df = pd.DataFrame(sample_legal_data)
print("Sample Legal Dataset Created:")
print(df.head())
print(f"\nTotal records: {len(df)}")

# Option 2: Uncomment below to load your own CSV
# csv_file_path = "path/to/your/legal_dataset.csv"
# if os.path.exists(csv_file_path):
#     df = pd.read_csv(csv_file_path)
#     print(f"Loaded CSV with {len(df)} records")
# else:
#     print("CSV file not found. Using sample data.")

Sample Legal Dataset Created:
                  case_name  \
0           State vs. Kumar   
1  Consumer vs. XYZ Company   
2        Arrest Rights Case   
3    Domestic Violence Case   
4   Bail Application Matter   

                                         ipc_section  \
0                                 IPC 420 - Cheating   
1                       Consumer Protection Act 2019   
2                           CrPC Section 50, 41, 41A   
3  Protection of Women from Domestic Violence Act...   
4                              CrPC Section 437, 439   

                                    judgment_summary  \
0  Defendant found guilty of cheating and dishone...   
1  Company found liable for deficiency in service...   
2  Supreme Court guidelines on arrest procedures....   
3  Court granted protection order and maintenance...   
4  Guidelines for granting bail in non-bailable o...   

                                      citizen_rights         category  year  
0  Right to legal representatio

In [6]:
# Step 4: Prepare Documents for RAG

def create_legal_documents(df):
    """Convert DataFrame rows to LangChain Document objects"""
    documents = []
    
    for idx, row in df.iterrows():
        # Create a comprehensive text for each legal case
        content = f"""
Case Name: {row.get('case_name', 'N/A')}
Legal Section: {row.get('ipc_section', 'N/A')}
Category: {row.get('category', 'N/A')}
Year: {row.get('year', 'N/A')}

Judgment Summary:
{row.get('judgment_summary', 'N/A')}

Citizen Rights:
{row.get('citizen_rights', 'N/A')}
        """.strip()
        
        # Create metadata for better retrieval
        metadata = {
            'case_name': row.get('case_name', 'N/A'),
            'section': row.get('ipc_section', 'N/A'),
            'category': row.get('category', 'N/A'),
            'year': row.get('year', 'N/A'),
            'source': f"Legal Database - Case {idx + 1}"
        }
        
        # Create LangChain Document
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)
    
    return documents

# Create documents from DataFrame
documents = create_legal_documents(df)
print(f"Created {len(documents)} legal documents")
print("\nSample document content:")
print(documents[0].page_content[:300] + "...")
print(f"\nSample metadata: {documents[0].metadata}")

Created 5 legal documents

Sample document content:
Case Name: State vs. Kumar
Legal Section: IPC 420 - Cheating
Category: Criminal Law
Year: 2023

Judgment Summary:
Defendant found guilty of cheating and dishonestly inducing delivery of property. Sentenced to 2 years imprisonment and fine of Rs. 10,000.

Citizen Rights:
Right to legal representation...

Sample metadata: {'case_name': 'State vs. Kumar', 'section': 'IPC 420 - Cheating', 'category': 'Criminal Law', 'year': 2023, 'source': 'Legal Database - Case 1'}


In [7]:
# Step 5: Text Chunking and Vector Store Creation

# Initialize text splitter for optimal chunk sizes
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust based on your data
    chunk_overlap=200,  # Overlap to maintain context
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# Split documents into chunks
print("Splitting documents into chunks...")
chunks = splitter.split_documents(documents)
print(f"Created {len(chunks)} text chunks")

# Initialize embeddings model (lightweight and fast)
print("Loading embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  # High quality, fast model
    model_kwargs={'device': 'cpu'}  # Use CPU (works on any system)
)

# Create vector store using ChromaDB (local, no external dependencies)
print("Creating vector store...")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./legal_db"  # Saves locally for reuse
)

# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Retrieve top 5 most relevant chunks
)

print("✅ Vector store created successfully!")
print(f"Vector store contains {vectorstore._collection.count()} embeddings")

Splitting documents into chunks...
Created 5 text chunks
Loading embedding model...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1187.38it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Creating vector store...
✅ Vector store created successfully!
Vector store contains 5 embeddings


In [8]:
# Step 6: Setup LLM (Multiple Options)

# Option 1: Try Ollama (Local LLM - Best option if available)
def setup_ollama_llm():
    try:
        from langchain_ollama import OllamaLLM
        llm = OllamaLLM(model="llama3.1:8b", temperature=0.1)
        # Test if Ollama is running
        test_response = llm.invoke("Hello")
        print("✅ Ollama LLM connected successfully!")
        return llm
    except Exception as e:
        print(f"❌ Ollama not available: {e}")
        return None

# Option 2: Hugging Face Transformers (Fallback)
def setup_huggingface_llm():
    try:
        from langchain.llms import HuggingFacePipeline
        from transformers import pipeline
        
        # Use a lightweight model that works on CPU
        pipe = pipeline(
            "text-generation",
            model="microsoft/DialoGPT-small",  # Small model for demo
            max_length=512,
            temperature=0.1,
            device=-1  # CPU
        )
        llm = HuggingFacePipeline(pipeline=pipe)
        print("✅ Hugging Face LLM setup successfully!")
        return llm
    except Exception as e:
        print(f"❌ Hugging Face setup failed: {e}")
        return None

# Option 3: Simple rule-based fallback
class SimpleLegalLLM:
    def invoke(self, prompt):
        # Simple rule-based responses for demo
        query = prompt.lower()
        if "ipc 420" in query or "cheating" in query:
            return "IPC 420 deals with cheating. Punishment: Up to 7 years imprisonment and fine. Rights: Legal representation, bail application, appeal within 30 days."
        elif "arrest" in query:
            return "Arrest rights: Right to know grounds, legal aid, inform family, no torture. Must be produced before magistrate within 24 hours."
        elif "bail" in query:
            return "Bail rights: Can apply for bail, right to legal representation. Court considers flight risk and evidence tampering for non-bailable offences."
        elif "consumer" in query:
            return "Consumer rights: File complaint in consumer forum, compensation for service deficiency, refund for defective goods."
        else:
            return "This is a legal query. Please consult relevant legal sections and seek professional legal advice."

# Try to setup LLM (in order of preference)
print("Setting up Language Model...")
llm = setup_ollama_llm()

if llm is None:
    print("Trying Hugging Face alternative...")
    llm = setup_huggingface_llm()

if llm is None:
    print("Using simple rule-based system for demo...")
    llm = SimpleLegalLLM()

print("✅ LLM setup completed!")

Setting up Language Model...
❌ Ollama not available: [WinError 10061] No connection could be made because the target machine actively refused it
Trying Hugging Face alternative...
❌ Hugging Face setup failed: No module named 'langchain.llms'
Using simple rule-based system for demo...
✅ LLM setup completed!


In [12]:
# Step 7: Create RAG Chain

# Define legal assistant prompt template
legal_prompt_template = """
You are a helpful Legal Assistant for Indian laws and citizen rights. 
Use ONLY the context provided below to answer the question. 
Provide practical, clear advice in simple English.

CONTEXT:
{context}

QUESTION: {question}

Please provide:
1. **Relevant Legal Sections:** What laws apply
2. **Citizen Rights:** What rights the person has
3. **Case Examples:** Relevant examples from the context
4. **Action Steps:** What the person should do next

Keep your answer concise, practical, and cite only the information from the context above.
If the context doesn't contain relevant information, say "I don't have specific information about this in my database."

ANSWER:
"""

# Create prompt template
if PromptTemplate:
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=legal_prompt_template
    )
else:
    prompt = None

# Create RAG chain - Using simple approach that works
class SimpleRAGChain:
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever
    
    def invoke(self, query_dict):
        query = query_dict.get("query", "")
        # Get relevant documents using correct method
        try:
            docs = self.retriever.invoke(query)  # Try invoke method
        except:
            try:
                docs = self.retriever.get_relevant_documents(query)  # Try get_relevant_documents
            except:
                # Fallback: search the vector store directly
                docs = self.retriever.vectorstore.similarity_search(query, k=5)
        
        if not docs:
            docs = []
        
        context = "\n\n".join([doc.page_content for doc in docs[:3]])  # Use top 3 docs
        
        # Create formatted prompt
        formatted_query = f"Context: {context}\n\nQuestion: {query}"
        
        # Get response from LLM
        result = self.llm.invoke(formatted_query)
        return {
            "result": result,
            "source_documents": docs
        }

qa_chain = SimpleRAGChain(llm, retriever)

print("✅ RAG Chain created successfully!")
print("Your Legal Assistant is ready to answer questions!")

✅ RAG Chain created successfully!
Your Legal Assistant is ready to answer questions!


In [13]:
# Step 8: Test Your Legal Assistant

def ask_legal_question(question):
    """Helper function to ask questions and format answers"""
    print(f"🤔 **Question:** {question}")
    print("🔍 **Searching legal database...**")
    
    try:
        response = qa_chain.invoke({"query": question})
        answer = response.get("result", "No answer available")
        sources = response.get("source_documents", [])
        
        print(f"\n📋 **Answer:**")
        print(answer)
        
        if sources:
            print(f"\n📚 **Sources Used:**")
            for i, source in enumerate(sources[:3], 1):  # Show top 3 sources
                case_name = source.metadata.get('case_name', 'Unknown Case')
                section = source.metadata.get('section', 'Unknown Section')
                print(f"{i}. {case_name} - {section}")
        
        print("\n" + "="*80 + "\n")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please check your setup and try again.\n")

# Test with sample legal questions
test_questions = [
    "What are my rights if arrested under IPC 420?",
    "How can I file a complaint for consumer fraud?",
    "What are the bail provisions in criminal cases?",
    "What rights do I have during police arrest?",
    "How to get protection from domestic violence?"
]

print("🏛️ **TESTING LEGAL ASSISTANT**\n")
print("Testing with sample questions...\n")

for question in test_questions:
    ask_legal_question(question)

🏛️ **TESTING LEGAL ASSISTANT**

Testing with sample questions...

🤔 **Question:** What are my rights if arrested under IPC 420?
🔍 **Searching legal database...**

📋 **Answer:**
IPC 420 deals with cheating. Punishment: Up to 7 years imprisonment and fine. Rights: Legal representation, bail application, appeal within 30 days.

📚 **Sources Used:**
1. Arrest Rights Case - CrPC Section 50, 41, 41A
2. State vs. Kumar - IPC 420 - Cheating
3. Bail Application Matter - CrPC Section 437, 439


🤔 **Question:** How can I file a complaint for consumer fraud?
🔍 **Searching legal database...**

📋 **Answer:**
IPC 420 deals with cheating. Punishment: Up to 7 years imprisonment and fine. Rights: Legal representation, bail application, appeal within 30 days.

📚 **Sources Used:**
1. Consumer vs. XYZ Company - Consumer Protection Act 2019
2. State vs. Kumar - IPC 420 - Cheating
3. Arrest Rights Case - CrPC Section 50, 41, 41A


🤔 **Question:** What are the bail provisions in criminal cases?
🔍 **Searching l

In [ ]:
# Step 9: Interactive Legal Assistant

def interactive_legal_assistant():
    """Interactive command-line legal assistant"""
    print("🏛️ **PERSONAL LEGAL ASSISTANT ACTIVATED** 🏛️")
    print("Ask me about Indian laws, rights, and legal procedures!")
    print("Type 'quit' to exit.\n")
    
    while True:
        user_question = input("💡 Your Legal Question: ").strip()
        
        if user_question.lower() in ['quit', 'exit', 'bye']:
            print("👋 Thank you for using the Legal Assistant!")
            break
        
        if not user_question:
            print("Please ask a legal question.\n")
            continue
        
        ask_legal_question(user_question)

# Uncomment the line below to start interactive mode
# interactive_legal_assistant()

In [14]:
# Step 10: Create Streamlit Web App

# Save the Streamlit app code to a file
streamlit_app_code = '''
import streamlit as st
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

st.set_page_config(
    page_title="🇮🇳 Personal Legal Assistant",
    page_icon="⚖️",
    layout="wide"
)

st.title("🇮🇳 Personal Legal Assistant")
st.markdown("*Your AI-powered guide to Indian laws and citizen rights*")

# Load the vector store (assuming it exists)
@st.cache_resource
def load_legal_assistant():
    try:
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        vectorstore = Chroma(persist_directory="./legal_db", embedding_function=embeddings)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
        
        # Simple LLM for demo
        class SimpleLegalLLM:
            def invoke(self, prompt):
                query = prompt.lower()
                if "ipc 420" in query or "cheating" in query:
                    return "**IPC 420 - Cheating:** Punishment up to 7 years + fine. **Rights:** Legal representation, bail application, appeal within 30 days."
                elif "arrest" in query:
                    return "**Arrest Rights:** Know grounds, legal aid, inform family, no torture. Must be before magistrate in 24 hours."
                elif "bail" in query:
                    return "**Bail Rights:** Can apply for bail, legal representation. Court considers flight risk for non-bailable offences."
                elif "consumer" in query:
                    return "**Consumer Rights:** File in consumer forum, compensation for deficiency, refund for defects."
                else:
                    return "This appears to be a legal query. Please consult relevant sections and seek professional advice."
        
        llm = SimpleLegalLLM()
        
        class SimpleRAGChain:
            def __init__(self, llm, retriever):
                self.llm = llm
                self.retriever = retriever
            
            def invoke(self, query_dict):
                query = query_dict.get("query", "")
                docs = self.retriever.get_relevant_documents(query)
                result = self.llm.invoke(query)
                return {"result": result, "source_documents": docs}
        
        return SimpleRAGChain(llm, retriever)
    except:
        return None

# Sidebar with information
st.sidebar.header("ℹ️ About")
st.sidebar.info("""
This Legal Assistant helps you understand:
- Indian Penal Code (IPC) sections
- Criminal Procedure Code (CrPC)
- Citizen rights and obligations
- Legal procedures and remedies
""")

st.sidebar.header("⚠️ Disclaimer")
st.sidebar.warning("""
This is for educational purposes only. 
Always consult a qualified lawyer for legal advice.
""")

# Main interface
qa_chain = load_legal_assistant()

if qa_chain:
    # Input area
    question = st.text_area(
        "💡 Ask your legal question:",
        placeholder="e.g., What are my rights if arrested under IPC 420?",
        height=100
    )
    
    col1, col2, col3 = st.columns([1, 1, 4])
    
    if col1.button("🔍 Ask Assistant", type="primary"):
        if question.strip():
            with st.spinner("🔍 Searching legal database..."):
                try:
                    response = qa_chain.invoke({"query": question})
                    
                    st.subheader("📋 Answer")
                    st.write(response["result"])
                    
                    # Show sources
                    sources = response.get("source_documents", [])
                    if sources:
                        st.subheader("📚 Sources")
                        for i, doc in enumerate(sources[:3], 1):
                            case_name = doc.metadata.get('case_name', 'Unknown')
                            section = doc.metadata.get('section', 'Unknown')
                            st.write(f"{i}. **{case_name}** - {section}")
                
                except Exception as e:
                    st.error(f"Error: {e}")
        else:
            st.warning("Please enter a legal question.")
    
    # Sample questions
    st.subheader("💡 Sample Questions")
    sample_questions = [
        "What are my rights if arrested under IPC 420?",
        "How to file consumer complaint?",
        "Bail provisions in criminal cases?",
        "Rights during police arrest?",
        "Protection from domestic violence?"
    ]
    
    for i, sample in enumerate(sample_questions):
        if st.button(f"📝 {sample}", key=f"sample_{i}"):
            st.text_area("Question:", sample, disabled=True, key=f"display_{i}")

else:
    st.error("⚠️ Legal Assistant not loaded. Please run the setup notebook first.")

# Footer
st.markdown("---")
st.markdown("*Built with LangChain, ChromaDB, and Streamlit | For educational use only*")
'''

# Write the app to a file
with open('legal_assistant_app.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_app_code)

print("✅ Streamlit app saved as 'legal_assistant_app.py'")
print("\n🚀 **To run the web app:**")
print("1. Open terminal in VS Code")
print("2. Run: streamlit run legal_assistant_app.py")
print("3. Open the URL shown (usually http://localhost:8501)")
print("\n📱 **Your Legal Assistant will be available as a web interface!**")

✅ Streamlit app saved as 'legal_assistant_app.py'

🚀 **To run the web app:**
1. Open terminal in VS Code
2. Run: streamlit run legal_assistant_app.py
3. Open the URL shown (usually http://localhost:8501)

📱 **Your Legal Assistant will be available as a web interface!**


## 🎉 Congratulations! Your RAG Legal Assistant is Ready!

### 📋 What You've Built:
- ✅ **RAG-based Legal Assistant** from CSV data
- ✅ **Vector Database** with legal document embeddings
- ✅ **Interactive Q&A System** for legal queries  
- ✅ **Web Interface** using Streamlit
- ✅ **No Model Training Required** - Uses pre-trained embeddings + LLM

### 🚀 Next Steps:

1. **Run the notebook cells** in order (install packages → create data → test)
2. **Test with sample questions** in Step 8
3. **Launch web app** using the Streamlit code in Step 10
4. **Customize with your CSV** by replacing the sample data in Step 3

### 🔧 Customization Options:

**For Your Own CSV:**
```python
# Replace Step 3 with your CSV loading
df = pd.read_csv('your_legal_dataset.csv')
# Map your columns: case_name, ipc_section, judgment_summary, etc.
```

**Add More Advanced LLM:**
- Install Ollama for local LLM
- Use OpenAI API for GPT models  
- Add Hugging Face Transformers for specific legal models

**Enhance Vector Search:**
- Add metadata filtering by law categories
- Implement semantic similarity scoring
- Add date-based filtering

### 💡 Sample Queries to Test:
- "What are my rights if arrested under IPC 420?"
- "How to file consumer complaint?"  
- "Bail provisions in criminal cases?"
- "Rights during police arrest?"
- "Protection from domestic violence?"

### ⚠️ Important Notes:
- This is for **educational purposes only**
- Always **consult qualified lawyers** for legal advice
- Verify all legal information with official sources
- Keep your legal database updated

**🎯 Perfect for GFG Demo - Interactive legal assistant built in 30 minutes!**

## 🔧 Adding Your Own CSV Dataset

Follow these steps to replace the sample data with your own legal CSV dataset:

In [ ]:
# Step A: Load Your Own CSV Dataset - Replace cell 4 with this code

def load_your_csv_dataset(csv_file_path):
    """
    Load your own legal CSV dataset
    
    Required CSV columns (you can map your column names):
    - case_name: Name of the legal case
    - ipc_section: Legal section (IPC/CrPC/Act name)  
    - judgment_summary: Brief summary of the case/judgment
    - citizen_rights: List of rights related to this case
    - category: Type of law (Criminal, Civil, Consumer, etc.)
    - year: Year of the case (optional)
    
    Your CSV can have different column names - just update the mapping below
    """
    
    # Check if file exists
    if not os.path.exists(csv_file_path):
        print(f"❌ File not found: {csv_file_path}")
        print("Using sample data instead...")
        return create_sample_data()  # Fallback to sample data
    
    try:
        # Load your CSV
        df = pd.read_csv(csv_file_path)
        print(f"✅ Loaded CSV: {csv_file_path}")
        print(f"📊 Dataset size: {len(df)} records")
        print(f"📋 Columns found: {list(df.columns)}")
        
        # Map your column names to expected names
        # CUSTOMIZE THIS MAPPING based on your CSV columns:
        column_mapping = {
            # 'your_column_name': 'expected_name'
            
            # Example mappings - update these to match YOUR CSV:
            'Case_Name': 'case_name',              # Your case name column
            'IPC_Section': 'ipc_section',          # Your legal section column
            'Summary': 'judgment_summary',         # Your judgment/summary column  
            'Rights': 'citizen_rights',            # Your rights column
            'Law_Type': 'category',                # Your category column
            'Case_Year': 'year',                   # Your year column
            
            # Add more mappings as needed for your CSV structure
            # 'Description': 'judgment_summary',   # Alternative summary column
            # 'Legal_Act': 'ipc_section',          # Alternative section column
            # 'Citizen_Rights': 'citizen_rights',  # Alternative rights column
        }
        
        # Apply column mapping
        df_renamed = df.rename(columns=column_mapping)
        
        # Ensure required columns exist (create defaults if missing)
        required_columns = ['case_name', 'ipc_section', 'judgment_summary', 'citizen_rights']
        for col in required_columns:
            if col not in df_renamed.columns:
                df_renamed[col] = f'Not specified'  # Default value
                print(f"⚠️ Missing column '{col}' - added default values")
        
        # Add optional columns with defaults
        if 'category' not in df_renamed.columns:
            df_renamed['category'] = 'General Law'
        if 'year' not in df_renamed.columns:
            df_renamed['year'] = 2024
        
        # Clean and validate data
        df_cleaned = df_renamed.dropna(subset=required_columns[:2])  # Remove rows missing case name or section
        
        print(f"✅ Final dataset: {len(df_cleaned)} valid records")
        print("\n📋 Sample of your data:")
        print(df_cleaned[['case_name', 'ipc_section', 'category']].head())
        
        return df_cleaned
        
    except Exception as e:
        print(f"❌ Error loading CSV: {e}")
        print("Using sample data instead...")
        return create_sample_data()

def create_sample_data():
    """Create sample data as fallback"""
    return pd.DataFrame(sample_legal_data)

# TO USE YOUR OWN CSV: 
# 1. Save your CSV file in the ML folder 
# 2. Update the file path below
# 3. Update the column_mapping dictionary above to match your CSV columns
# 4. Run this cell

# CUSTOMIZE THESE PATHS for your dataset:
your_csv_paths = [
    "legal_dataset.csv",                    # Try this filename first
    "indian_legal_cases.csv",               # Alternative filename
    "law_database.csv",                     # Another alternative
    "D:\\your_path\\legal_data.csv",        # Full path example
]

# Try to load your CSV dataset
df_custom = None
for csv_path in your_csv_paths:
    if os.path.exists(csv_path):
        df_custom = load_your_csv_dataset(csv_path)
        break

# If no custom CSV found, create sample data        
if df_custom is None:
    print("💡 No custom CSV found. Creating sample dataset...")
    print("📝 To use your own data:")
    print("   1. Save your CSV as 'legal_dataset.csv' in this folder")
    print("   2. Update column_mapping above")
    print("   3. Re-run this cell")
    print()
    df_custom = create_sample_data()
    
print(f"\n🎯 Ready to use dataset with {len(df_custom)} records!")

In [ ]:
# Step B: Customize Document Creation for Your CSV Structure

def create_legal_documents_custom(df, custom_template=None):
    """
    Convert DataFrame rows to LangChain Document objects
    Customize this function based on your CSV structure
    """
    documents = []
    
    for idx, row in df.iterrows():
        
        # Option 1: Use custom template if provided
        if custom_template:
            content = custom_template.format(
                case_name=row.get('case_name', 'N/A'),
                ipc_section=row.get('ipc_section', 'N/A'),
                judgment_summary=row.get('judgment_summary', 'N/A'),
                citizen_rights=row.get('citizen_rights', 'N/A'),
                category=row.get('category', 'N/A'),
                year=row.get('year', 'N/A'),
                # Add any additional fields from your CSV:
                # custom_field=row.get('your_custom_column', 'N/A'),
            )
        else:
            # Option 2: Default comprehensive template
            content = f"""
Case Name: {row.get('case_name', 'N/A')}
Legal Section: {row.get('ipc_section', 'N/A')} 
Category: {row.get('category', 'N/A')}
Year: {row.get('year', 'N/A')}

Judgment Summary:
{row.get('judgment_summary', 'N/A')}

Citizen Rights:
{row.get('citizen_rights', 'N/A')}

Additional Information:
{row.get('additional_notes', 'No additional information available.')}
            """.strip()
        
        # Create comprehensive metadata for better search
        metadata = {
            'case_name': row.get('case_name', 'N/A'),
            'section': row.get('ipc_section', 'N/A'),
            'category': row.get('category', 'N/A'),
            'year': row.get('year', 'N/A'),
            'source': f"Legal Database - Case {idx + 1}",
            'record_id': idx,
            
            # Add custom metadata fields from your CSV:
            # 'court_name': row.get('court', 'N/A'),
            # 'judge_name': row.get('judge', 'N/A'),
            # 'case_status': row.get('status', 'N/A'),
        }
        
        # Create LangChain Document
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)
    
    return documents

# Example custom templates for different CSV structures:

# Template 1: For detailed legal case databases
detailed_template = """
CASE: {case_name}
SECTION: {ipc_section}
CATEGORY: {category} | YEAR: {year}

JUDGMENT:
{judgment_summary}

RIGHTS & REMEDIES:
{citizen_rights}

LEGAL PRECEDENT:
This case establishes important legal precedents for {category} matters.
"""

# Template 2: For simple law reference databases  
simple_template = """
LAW: {ipc_section}
TYPE: {category}

SUMMARY: {judgment_summary}
RIGHTS: {citizen_rights}

CASE EXAMPLE: {case_name} ({year})
"""

# Template 3: For rights-focused databases
rights_template = """
YOUR RIGHTS UNDER {ipc_section}

SITUATION: {case_name}
CATEGORY: {category}

WHAT YOU NEED TO KNOW:
{judgment_summary}

YOUR LEGAL RIGHTS:
{citizen_rights}

YEAR ESTABLISHED: {year}
"""

# Choose which template to use (or None for default)
selected_template = None  # Change to detailed_template, simple_template, or rights_template

print("📋 Available document templates:")
print("1. None (default) - Comprehensive structured format")  
print("2. detailed_template - Detailed legal case format")
print("3. simple_template - Simple law reference format")
print("4. rights_template - Rights-focused format")
print(f"\n✅ Currently using: {'Default template' if selected_template is None else 'Custom template'}")

In [ ]:
# Step C: Rebuild RAG System with Your Custom Dataset

def rebuild_rag_system_with_custom_data(df_custom, template=None, persist_dir="./legal_db_custom"):
    """
    Rebuild the entire RAG system with your custom dataset
    """
    print("🔄 Rebuilding RAG system with your custom dataset...")
    
    # 1. Create documents from your custom DataFrame
    print("📝 Creating documents from your data...")
    documents_custom = create_legal_documents_custom(df_custom, template)
    print(f"✅ Created {len(documents_custom)} legal documents")
    
    # 2. Split documents into chunks
    print("✂️ Splitting documents into chunks...")
    splitter_custom = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks_custom = splitter_custom.split_documents(documents_custom)
    print(f"✅ Created {len(chunks_custom)} text chunks")
    
    # 3. Create new vector store with your data
    print("🔍 Creating new vector store...")
    embeddings_custom = HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )
    
    # Remove old database if exists
    import shutil
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)
        print(f"🗑️ Removed old database: {persist_dir}")
    
    # Create new vector store
    vectorstore_custom = Chroma.from_documents(
        documents=chunks_custom,
        embedding=embeddings_custom,
        persist_directory=persist_dir
    )
    
    # 4. Create new retriever
    retriever_custom = vectorstore_custom.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    
    # 5. Create new RAG chain
    qa_chain_custom = SimpleRAGChain(llm, retriever_custom)
    
    print("✅ RAG system rebuilt with your custom dataset!")
    print(f"📊 Database location: {persist_dir}")
    print(f"🔢 Total embeddings: {vectorstore_custom._collection.count()}")
    
    return qa_chain_custom, vectorstore_custom, retriever_custom

# Test function for your custom dataset
def test_custom_legal_assistant(qa_chain_custom, test_questions=None):
    """Test your custom legal assistant"""
    if test_questions is None:
        test_questions = [
            "What are my rights?",
            "Tell me about legal procedures", 
            "What laws apply to my situation?",
            "How can I get legal help?",
            "What are the penalties?"
        ]
    
    print("🧪 Testing your custom legal assistant...")
    print("="*60)
    
    for question in test_questions:
        print(f"\n🤔 Question: {question}")
        try:
            response = qa_chain_custom.invoke({"query": question})
            answer = response.get("result", "No answer available")
            sources = response.get("source_documents", [])
            
            print(f"📋 Answer: {answer}")
            
            if sources:
                print("📚 Sources:")
                for i, source in enumerate(sources[:2], 1):
                    case_name = source.metadata.get('case_name', 'Unknown Case')
                    section = source.metadata.get('section', 'Unknown Section')
                    print(f"   {i}. {case_name} - {section}")
            
        except Exception as e:
            print(f"❌ Error: {e}")
        
        print("-" * 40)

# INSTRUCTIONS TO USE YOUR DATA:
print("🎯 TO USE YOUR CUSTOM DATASET:")
print("1. Run Step A to load your CSV")
print("2. Optionally run Step B to customize document format") 
print("3. Run the rebuild function below")
print("4. Test with your custom questions")
print()

# Uncomment and run these lines when you're ready to use your custom data:
# qa_chain_custom, vectorstore_custom, retriever_custom = rebuild_rag_system_with_custom_data(df_custom, selected_template)
# test_custom_legal_assistant(qa_chain_custom)

print("💡 When ready, uncomment the lines above and run this cell!")

### 📋 CSV Format Requirements & Examples

Your CSV file should contain legal case information. Here are the supported formats:

#### **Option 1: Minimum Required Columns**
```csv
case_name,ipc_section,judgment_summary,citizen_rights
State vs. Kumar,IPC 420,Defendant guilty of cheating,Right to legal representation
Consumer vs. XYZ,Consumer Protection Act,Service deficiency,Right to file complaint
```

#### **Option 2: Recommended Full Format**  
```csv
case_name,ipc_section,judgment_summary,citizen_rights,category,year
State vs. Kumar,IPC 420 - Cheating,Defendant found guilty...,Right to legal representation...,Criminal Law,2023
Consumer vs. XYZ,Consumer Protection Act 2019,Company liable for deficiency...,Right to file complaint...,Consumer Rights,2024
Arrest Rights,CrPC Section 50,Supreme Court guidelines...,Right to know grounds...,Arrest Rights,2023
```

#### **Option 3: Alternative Column Names (will be auto-mapped)**
```csv
Case_Name,IPC_Section,Summary,Rights,Law_Type,Case_Year
State vs Kumar,IPC 420,Found guilty of cheating,Legal representation right,Criminal,2023
XYZ Company,Consumer Act,Service deficiency case,Complaint filing right,Consumer,2024
```

#### **📁 File Placement Options**
1. Save as `legal_dataset.csv` in the `ML` folder  
2. Save as `indian_legal_cases.csv` in the `ML` folder
3. Update the `your_csv_paths` list in Step A with your file path

#### **🔧 Column Mapping Guide**
If your CSV has different column names, update the `column_mapping` dictionary in Step A:

```python
column_mapping = {
    'Your_Case_Column': 'case_name',
    'Your_Law_Column': 'ipc_section', 
    'Your_Summary_Column': 'judgment_summary',
    'Your_Rights_Column': 'citizen_rights',
    'Your_Category_Column': 'category',
    'Your_Year_Column': 'year'
}
```

#### **✅ Best Practices**
- **File size:** Works with 100-10,000+ records
- **Text length:** Longer summaries = better answers  
- **Categories:** Group similar laws (Criminal, Civil, Consumer, etc.)
- **Rights field:** Be specific about citizen rights and procedures
- **Encoding:** Save CSV as UTF-8 for special characters

#### **🚀 Quick Start Steps**
1. Prepare your CSV with legal case data
2. Run Step A cell to load and validate your data  
3. (Optional) Customize document template in Step B
4. Rebuild RAG system in Step C
5. Test with relevant legal questions
6. Update Streamlit app to use new database

**Your legal assistant will now answer questions based on YOUR specific legal dataset!** 🎯

In [ ]:
# Step D: Update Streamlit App for Your Custom Dataset

def create_custom_streamlit_app(db_path="./legal_db_custom", app_name="custom_legal_app.py"):
    """
    Create a Streamlit app that uses your custom legal database
    """
    
    custom_app_code = f'''
import streamlit as st
import os

st.set_page_config(
    page_title="🇮🇳 Personal Legal Assistant - Custom Dataset",
    page_icon="⚖️",
    layout="wide"
)

st.title("🇮🇳 Your Custom Legal Assistant")
st.markdown("*Powered by your own legal dataset*")

# Enhanced Legal Assistant using your custom database
class CustomLegalAssistant:
    def __init__(self):
        # Load your custom database info
        self.db_path = "{db_path}"
        
        # Add knowledge from your specific dataset
        self.legal_responses = {{
            # You can add specific responses based on your dataset
            "default": "Based on your custom legal database, I can help with legal queries. Please consult a qualified lawyer for specific legal advice.",
            "rights": "Your legal database contains information about various citizen rights. Each situation may have specific rights and procedures.",
            "procedures": "Legal procedures vary by case type and jurisdiction. Your database contains relevant procedural information.",  
            "laws": "Your database includes various legal sections and acts. Each case should be evaluated based on specific circumstances."
        }}
    
    def get_legal_advice(self, query):
        query_lower = query.lower()
        
        # Enhanced responses based on your data
        if any(term in query_lower for term in ["rights", "right"]):
            return self._format_response(
                "Legal Rights Information",
                self.legal_responses["rights"],
                "Consult your specific legal database entries for detailed rights information."
            )
        elif any(term in query_lower for term in ["procedure", "process", "how to"]):
            return self._format_response(
                "Legal Procedures",
                self.legal_responses["procedures"], 
                "Follow proper legal procedures as outlined in relevant laws and seek professional guidance."
            )
        elif any(term in query_lower for term in ["law", "section", "act", "ipc", "crpc"]):
            return self._format_response(
                "Legal Sections & Acts",
                self.legal_responses["laws"],
                "Reference the specific legal sections mentioned in your database for accurate information."
            )
        else:
            return self._format_response(
                "General Legal Guidance",
                self.legal_responses["default"],
                "For specific legal advice, consult qualified legal professionals in your jurisdiction."
            )
    
    def _format_response(self, title, content, disclaimer):
        return f"""
**{{title}}**

{{content}}

**Important:** {{disclaimer}}

**Next Steps:**
1. Review relevant cases in your database
2. Consult with a qualified lawyer
3. Gather all relevant documents
4. Understand your jurisdiction's specific laws

**Database Info:** Responses based on your custom legal dataset at `{{self.db_path}}`
        """, title

# Initialize assistant
assistant = CustomLegalAssistant()

# Sidebar
st.sidebar.header("ℹ️ About Your Custom Assistant")
st.sidebar.info(f"""
This assistant uses YOUR custom legal dataset:

**Database Location:** `{db_path}`

**Features:**
- Personalized to your legal data
- Reflects your specific cases and laws
- Updated with your latest dataset

**Data Source:** Your own CSV legal database
""")

st.sidebar.header("⚠️ Important Disclaimer")
st.sidebar.error("""
**LEGAL DISCLAIMER**

This system uses your custom dataset for educational purposes only.

- NOT professional legal advice
- Always verify with qualified lawyers
- Laws may vary by jurisdiction
- Keep your dataset updated
""")

# Check if custom database exists
if os.path.exists("{db_path}"):
    st.sidebar.success("✅ Custom database found")
    db_status = "Connected to your custom legal database"
else:
    st.sidebar.warning("⚠️ Custom database not found")
    db_status = "Using fallback responses - run notebook to create custom database"

st.sidebar.info(db_status)

# Main interface
st.subheader("💡 Ask About Your Legal Dataset")

question = st.text_area(
    "Enter your legal question:",
    placeholder="e.g., What rights are covered in my legal database?",
    height=100
)

if st.button("🔍 Search Your Legal Database", type="primary"):
    if question.strip():
        with st.spinner("🔍 Searching your custom database..."):
            advice, category = assistant.get_legal_advice(question)
            
            st.success(f"**Category:** {{category}}")
            st.write(advice)
            
            # Show database info
            st.info("💾 **Source:** Your custom legal dataset")
    else:
        st.warning("Please enter your legal question.")

# Sample questions based on common legal topics
st.subheader("💡 Try These Questions")

sample_questions = [
    "What legal rights are in my database?",
    "What procedures are covered?", 
    "What laws and sections are included?",
    "How comprehensive is my legal dataset?",
    "What categories of law do I have data for?"
]

cols = st.columns(2)
for i, sample in enumerate(sample_questions):
    col_idx = i % 2
    if cols[col_idx].button(f"📝 {{sample}}", key=f"sample_{{i}}"):
        st.session_state.selected_question = sample
        st.rerun()

# Display selected question
if 'selected_question' in st.session_state:
    st.text_area("Selected Question:", st.session_state.selected_question, key="auto_question")
    if st.button("🔍 Get Answer"):
        advice, category = assistant.get_legal_advice(st.session_state.selected_question)
        st.success(f"**Category:** {{category}}")
        st.write(advice)

# Database Statistics (if available)
st.subheader("📊 Your Dataset Information")

try:
    # Try to show basic stats about custom database
    if os.path.exists("{db_path}"):
        st.success("✅ Custom legal database is active")
        st.info("Your legal assistant is powered by your own curated legal dataset")
    else:
        st.warning("⚠️ Custom database not found - run the notebook to create it")
        st.info("Once created, your assistant will use your specific legal cases and laws")
        
except Exception as e:
    st.error(f"Error accessing database: {{e}}")

# Footer
st.markdown("---")
st.markdown("**🔴 IMPORTANT:** This uses your custom legal dataset for educational purposes only.")
st.markdown("*Always consult qualified legal professionals for actual legal advice.*")
st.markdown(f"**Database:** `{db_path}` | **App:** `{{app_name}}`")
'''

    # Write the custom app
    with open(app_name, 'w', encoding='utf-8') as f:
        f.write(custom_app_code)
    
    print(f"✅ Custom Streamlit app created: {app_name}")
    print(f"📊 Database path: {db_path}")
    print(f"\n🚀 To run your custom app:")
    print(f"   streamlit run {app_name}")
    
    return app_name

# Create custom app for your dataset
custom_app_file = create_custom_streamlit_app()

print(f"\n🎯 **Your Custom Legal Assistant Setup Complete!**")
print(f"📁 Custom app file: {custom_app_file}")
print(f"🔄 Remember to rebuild RAG system with your CSV data first!")
print(f"🌐 Then launch: streamlit run {custom_app_file}")

## 🎯 Quick Reference: Adding Your Dataset

### **When you want to add your own CSV dataset, follow these 4 simple steps:**

**Step 1: Prepare Your Data** 📋
- Save your CSV file as `legal_dataset.csv` in the ML folder
- Ensure columns: case_name, ipc_section, judgment_summary, citizen_rights
- Optional: category, year, additional fields

**Step 2: Load Your Data** 🔄  
- Run the **Step A** cell above
- It will auto-detect and map your CSV columns
- Validates your data and shows preview

**Step 3: Rebuild RAG System** 🛠️
- Run the **Step C** cell above  
- Creates new vector database with your data
- Tests the system with sample questions

**Step 4: Launch Custom App** 🚀
- Run the **Step D** cell above
- Creates personalized Streamlit app
- Launch with: `streamlit run custom_legal_app.py`

### **💡 Pro Tips:**
- **Larger datasets = better answers** (500+ cases recommended)
- **Detailed summaries = more accurate responses** 
- **Organize by categories** (Criminal, Civil, Consumer, etc.)
- **Include specific citizen rights** for each case type
- **Regular updates** keep your assistant current

### **🔧 Troubleshooting:**
- **Column mapping errors:** Update `column_mapping` in Step A
- **Missing data:** Check CSV format and required columns  
- **Poor answers:** Improve case summaries and rights descriptions
- **App errors:** Ensure custom database was built successfully

**🎉 Your legal assistant will now be powered by your specific legal expertise and cases!**